Housing Costs + Property Taxes over 5-year span (2018-2022)

In [56]:
import requests
import pandas as pd
import csv

def process_census_data(file_path):
    census_data_dict = {}
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)
        for row in csv_reader:
            if len(row) == 2:
                census_data_dict[row[0]] = row[1]
    return census_data_dict

def process_year_data(year):
    file_path = f'ACSST5Y{year}.S2506-Data.csv'
    meta_data_file_path = 'ACSST5Y2022.S2506-Column-Metadata.csv'

    df = pd.read_csv(file_path)
    census_data_dict = process_census_data(meta_data_file_path)

    df = df.rename(columns=census_data_dict)

    filtered_columns = [
        col for col in df.columns
        if "MONTHLY HOUSING COSTS" in col or "REAL ESTATE TAXES" in col or "Geography" in col or "Geographic Area Name" in col or "Median household income" in col
    ]

    df = df[filtered_columns]
    df = df.iloc[1:]
    df['Geographic Area Name'] = df['Geographic Area Name'].str.replace('ZCTA5', '', regex=False).str.strip()

    # Add a year column
    df['Year'] = year

    return df

# Process data for years 2018 to 2022
years = range(2018, 2023)
dfs = [process_year_data(year) for year in years]

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined dataframe
print(combined_df.head())

# combined_df.to_csv('combined_housing_data_2018_2022.csv', index=False)

# Print some information about the combined dataset
# print(f"\nTotal number of rows: {len(combined_df)}")
print(f"Years included: {combined_df['Year'].unique()}")
# print(f"\nColumns in the dataset:")
# for col in combined_df.columns:
#     print(col)

<ipython-input-56-b669cd43d2f4>:19: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-56-b669cd43d2f4>:19: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,1

        Geography Geographic Area Name  \
0  8600000US00601                00601   
1  8600000US00602                00602   
2  8600000US00603                00603   
3  8600000US00606                00606   
4  8600000US00610                00610   

  Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)  \
0                                                 85                                                                                                                                                                     
1                                                538                                                                                                                                                                     
2                                                493                                         

Data Quality checks

In [57]:
combined_df.isna().sum().sum() / (combined_df.shape[0] * combined_df.shape[1])

0.010241686167461773

In [58]:
total_nan = combined_df.isna().sum().sum()

nan_percentage = combined_df.isna().mean() * 100


nan_percentage_sorted = nan_percentage.sort_values(ascending=False)


In [59]:
combined_df = combined_df.loc[:, ~combined_df.columns.str.contains('Margin of Error', case=False)]

In [60]:
# combined_df.sample(10).to_csv('housing_costs_sample.csv')

In [61]:
import pandas as pd
import numpy as np

def process_housing_costs(df):

    columns_to_keep = [
        'Geographic Area Name',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!Median (dollars)',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!Less than $200',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$200 to $399',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$400 to $599',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$600 to $799',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$800 to $999',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$1,000 to $1,499',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$1,500 to $1,999',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$2,000 to $2,499',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$2,500 to $2,999',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!$3,000 or more',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars)',
        'Year'
    ]

    df_selected = df[columns_to_keep]

    new_column_names = {
        'Geographic Area Name': 'Area',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)': 'Median_Income',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!MONTHLY HOUSING COSTS!!Median (dollars)': 'Median_Monthly_Cost',
        'Estimate!!Owner-occupied housing units with a mortgage!!Owner-occupied housing units with a mortgage!!REAL ESTATE TAXES!!Median (dollars)': 'Median_RE_Taxes'
    }
    df_selected.rename(columns=new_column_names, inplace=True)

    # Convert columns to numeric, replacing '-' with NaN
    numeric_columns = df_selected.columns.drop(['Area', 'Year'])
    for col in numeric_columns:
        df_selected[col] = pd.to_numeric(df_selected[col].replace('-', np.nan), errors='coerce')

    # Calculate derived features
    df_selected['Housing_Cost_Less_1000'] = df_selected.iloc[:, 3:8].sum(axis=1)
    df_selected['Housing_Cost_1000_1999'] = df_selected.iloc[:, 8:10].sum(axis=1)
    df_selected['Housing_Cost_2000_2999'] = df_selected.iloc[:, 10:12].sum(axis=1)
    df_selected['Housing_Cost_3000_Plus'] = df_selected.iloc[:, 12]

    # Calculate affordability ratio
    df_selected['Affordability_Ratio'] = df_selected['Median_Monthly_Cost'] / (df_selected['Median_Income'] / 12)

    # Select final columns
    final_columns = ['Area', 'Median_Income', 'Median_Monthly_Cost', 'Housing_Cost_Less_1000',
                     'Housing_Cost_1000_1999', 'Housing_Cost_2000_2999', 'Housing_Cost_3000_Plus',
                     'Median_RE_Taxes', 'Affordability_Ratio', 'Year']

    df_final = df_selected[final_columns]

    return df_final


processed_df = process_housing_costs(combined_df)

# Display some information about the processed dataset
display(f"\nProcessed dataset shape: {processed_df.shape}")
display("\nColumns in the processed dataset:")
display(processed_df.columns.tolist())
display("\nSample of the processed data:")
display(processed_df.head())

<ipython-input-61-d2012057ca9b>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns=new_column_names, inplace=True)
<ipython-input-61-d2012057ca9b>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[col] = pd.to_numeric(df_selected[col].replace('-', np.nan), errors='coerce')
<ipython-input-61-d2012057ca9b>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

'\nProcessed dataset shape: (166908, 10)'

'\nColumns in the processed dataset:'

['Area',
 'Median_Income',
 'Median_Monthly_Cost',
 'Housing_Cost_Less_1000',
 'Housing_Cost_1000_1999',
 'Housing_Cost_2000_2999',
 'Housing_Cost_3000_Plus',
 'Median_RE_Taxes',
 'Affordability_Ratio',
 'Year']

'\nSample of the processed data:'

,Area,Median_Income,Median_Monthly_Cost,Housing_Cost_Less_1000,Housing_Cost_1000_1999,Housing_Cost_2000_2999,Housing_Cost_3000_Plus,Median_RE_Taxes,Affordability_Ratio,Year
0,00601,85.0,92.0,514,28,0,801.0,NaN,12.988235,2018
1,00602,538.0,416.0,1783,233,0,877.0,NaN,9.278810,2018
2,00603,493.0,739.0,3169,402,51,839.0,NaN,17.987830,2018
3,00606,23.0,78.0,167,0,0,551.0,NaN,40.695652,2018
4,00610,462.0,426.0,1783,95,0,743.0,NaN,11.064935,2018
